In [ ]:
import torch, mmengine, mmcv, mmdet, mmdet3d, spconv
print(torch.__version__)
print(torch.version.cuda)
print(mmengine.__version__)
print(mmcv.__version__)
print(mmdet.__version__)
print(mmdet3d.__version__)
print(spconv.__version__)

In [11]:
import argparse
import logging
import os
import os.path as osp
import torch

from mmengine.config import Config, DictAction
from mmengine.logging import print_log
from mmengine.registry import RUNNERS
from mmengine.runner import Runner

from mmdet3d.utils import replace_ceph_backend

from projects.mmdet3d_plugin.models.detectors import CmtDetector
import time
from mmengine.structures import InstanceData

In [12]:
cfg = Config.fromfile('projects/configs/fusion/my_cmt.py')
# cfg = Config.fromfile('../mmdetection3d/projects/BEVFusion/configs/my_bevfusion.py')
cfg.work_dir = osp.abspath('./work_dirs')
runner = Runner.from_cfg(cfg)

03/16 18:32:14 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.18 | packaged by conda-forge | (default, Oct 10 2023, 15:44:36) [GCC 12.3.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 148932078
    GPU 0: NVIDIA GeForce RTX 3060
    CUDA_HOME: /usr/local/cuda-12.2
    NVCC: Cuda compilation tools, release 12.2, V12.2.91
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 2.1.2
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.1-Product Build 20230303 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencod

/media/helloalone/新加卷/ubuntu_anaconda_envs/python38/lib/python3.8/site-packages/mmengine/utils/manager.py:113: UserWarning: <class 'mmdet3d.visualization.local_visualizer.Det3DLocalVisualizer'> instance named of visualizer has been created, the method `get_instance` should not accept any other arguments
  warnings.warn(
/media/helloalone/新加卷/ubuntu_anaconda_envs/python38/lib/python3.8/site-packages/mmdet/models/task_modules/builder.py:17: UserWarning: ``build_sampler`` would be deprecated soon, please use ``mmdet.registry.TASK_UTILS.build()`` 
  warnings.warn('``build_sampler`` would be deprecated soon, please use '
/media/helloalone/新加卷/ubuntu_anaconda_envs/python38/lib/python3.8/site-packages/mmdet/models/task_modules/builder.py:39: UserWarning: ``build_assigner`` would be deprecated soon, please use ``mmdet.registry.TASK_UTILS.build()`` 
  warnings.warn('``build_assigner`` would be deprecated soon, please use '
/media/helloalone/新加卷/ubuntu_anaconda_envs/python38/lib/python3.8/site-p

03/16 18:32:17 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
03/16 18:32:17 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                

In [13]:
runner.model.train()

CmtDetector(
  (data_preprocessor): Det3DDataPreprocessor()
  (pts_voxel_encoder): HardSimpleVFE()
  (pts_middle_encoder): SparseEncoder(
    (conv_input): SparseSequential(
      (0): SubMConv3d(4, 16, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], bias=False, algo=ConvAlgo.MaskImplicitGemm)
      (1): BatchNorm1d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (encoder_layers): SparseSequential(
      (encoder_layer1): SparseSequential(
        (0): SparseBasicBlock(
          (conv1): SubMConv3d(16, 16, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], bias=False, algo=ConvAlgo.MaskImplicitGemm)
          (bn1): BatchNorm1d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (conv2): SubMConv3d(16, 16, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], ou

In [ ]:
# 训练过程
for data_batch in runner.train_dataloader:
    data_batch = runner.model.data_preprocessor(data_batch, training=True)
    if isinstance(data_batch, dict):
        losses = runner.model(**data_batch, mode='loss')
    elif isinstance(data_batch, (list, tuple)):
        losses = runner.model(*data_batch, mode='loss')
    else:
        raise TypeError()
    break

In [ ]:
losses

In [14]:
runner.model.eval()

CmtDetector(
  (data_preprocessor): Det3DDataPreprocessor()
  (pts_voxel_encoder): HardSimpleVFE()
  (pts_middle_encoder): SparseEncoder(
    (conv_input): SparseSequential(
      (0): SubMConv3d(4, 16, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], bias=False, algo=ConvAlgo.MaskImplicitGemm)
      (1): BatchNorm1d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (encoder_layers): SparseSequential(
      (encoder_layer1): SparseSequential(
        (0): SparseBasicBlock(
          (conv1): SubMConv3d(16, 16, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], output_padding=[0, 0, 0], bias=False, algo=ConvAlgo.MaskImplicitGemm)
          (bn1): BatchNorm1d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (conv2): SubMConv3d(16, 16, kernel_size=[3, 3, 3], stride=[1, 1, 1], padding=[1, 1, 1], dilation=[1, 1, 1], ou

In [ ]:
# 验证过程
for data_batch in runner.val_dataloader:
    data_batch = runner.model.data_preprocessor(data_batch, training=False)
    if isinstance(data_batch, dict):
        outputs = runner.model(**data_batch, mode='predict')
    elif isinstance(data_batch, (list, tuple)):
        outputs = runner.model(**data_batch, mode='predict')
    else:
        raise TypeError()
    runner.val_evaluator.process(data_samples=outputs, data_batch=data_batch)
    break
# with torch.no_grad():
#     metrics = runner.val_evaluator.evaluate(len(runner.val_dataloader.dataset))

In [15]:
runner.val_dataloader

03/16 18:32:22 - mmengine - INFO - ------------------------------
03/16 18:32:22 - mmengine - INFO - The length of test dataset: 81
03/16 18:32:22 - mmengine - INFO - The number of instances per category in the dataset:
+----------------------+--------+
| category             | number |
+----------------------+--------+
| car                  | 2568   |
| truck                | 124    |
| trailer              | 0      |
| bus                  | 41     |
| construction_vehicle | 0      |
| bicycle              | 52     |
| motorcycle           | 259    |
| pedestrian           | 1358   |
| traffic_cone         | 39     |
| barrier              | 0      |
+----------------------+--------+


In [ ]:
time_start = time.time()
with torch.no_grad():
    for data_batch in runner.val_dataloader:
        data_batch = runner.model.data_preprocessor(data_batch, training=False)
        if isinstance(data_batch, dict):
            outputs = runner.model(**data_batch, mode='predict')
        elif isinstance(data_batch, (list, tuple)):
            outputs = runner.model(**data_batch, mode='predict')
        else:
            raise TypeError()
print(time.time() - time_start)

In [ ]:
time_start = time.time()
with torch.no_grad():
    data_batch_raw = next(iter(runner.val_dataloader))
    for _ in range(100):
        data_batch = runner.model.data_preprocessor(data_batch_raw, training=False)
        if isinstance(data_batch, dict):
            outputs = runner.model(**data_batch, mode='predict')
        elif isinstance(data_batch, (list, tuple)):
            outputs = runner.model(**data_batch, mode='predict')
        else:
            raise TypeError()
print(time.time() - time_start)

In [17]:
with torch.no_grad():
    data_batch = next(iter(runner.val_dataloader))
    data_batch = runner.model.data_preprocessor(data_batch, training=False)
    batch_inputs_dict = data_batch['inputs']
    batch_data_samples = data_batch['data_samples']
    imgs = batch_inputs_dict.get('imgs', None)
    points = batch_inputs_dict.get('points', None)
    img_metas = [item.metainfo for item in batch_data_samples]

    time_start = time.time()
    for _ in range(100):
        img_feats = runner.model.extract_img_feat(imgs, img_metas)
    
    midedle1 = time.time()
    for _ in range(100):
        pts_feats = runner.model.extract_pts_feat(points, img_feats, img_metas)

    midedle2 = time.time()
    for _ in range(100):
        outs = runner.model.pts_bbox_head(pts_feats, img_feats, img_metas)

    middle3 = time.time()
    for _ in range(100):
        bbox_list = runner.model.pts_bbox_head.get_bboxes(
            outs, img_metas, rescale=False)
        
        # bbox_results = []
        # for bboxes, scores, labels in bbox_list:
        #     results = InstanceData()
        #     results.bboxes_3d = bboxes.to('cpu')
        #     results.scores_3d = scores.cpu()
        #     results.labels_3d = labels.cpu()
        #     bbox_results.append(results)
        # detsamples = runner.model.add_pred_to_datasample(batch_data_samples,
        #                                             data_instances_3d = bbox_results,
        #                                             data_instances_2d = None)
    end = time.time()
    print(midedle1 - time_start, midedle2 - midedle1, middle3 - midedle2, end - middle3)

4.977407217025757 3.253243923187256 3.5700490474700928 0.056304216384887695
